# Tutorial #1: Interfacing w/ the Library

**Target Audience:** Design Engineers / Front-end User

**Learning Objectives:**
1. Primary Functionality
    1. [Selecting a component](#selecting)
    2. [Meaning / notation of characteristics](#reading-characteristics)
    3. [Finding geometrical parameters which best suits target characteristics](#select-characteristics)
    4. [Export to `.gds` or dictionary compatable with `QComponent(options=...)`](#export)
2. Secondary Functionality
    1. Seeing who has contributed what data
    2. Seeing what was the simulation method of the contributed data


# Section 1: Primary Functionality

## Section 1.1: Selecting a Component <a id="selecting"></a>

First, let's see which components are currently supported.

In [1]:
import metal_library

print(metal_library.supported_components)

['TransmonCross', 'TransmonPocket']


Now we can place one of the supported components into `Reader`. This is a class meant to parse the library (located at `C:\\...\metal_library\library`)

In [2]:
from metal_library import Reader

reader = Reader(component_name='TransmonCross')
print(reader)

Interfacing with the library will be conducted through this object.

## Section 1.2: Understanding Structure of Characteristics <a id="reading-characteristics"></a>

Since we're conducting science, we must use exact language when talking about the characteristics we're pulling.

Example of **imprecise** language:
- 👎 "Get the coupling strength?"

Example of **precise** language:
- 👍 "Get the coupling strength between the readout cavity and qubit"
- 👍 "Get the coupling strength between qubit #1 and the bus (connecting qubit #1 and qubit #2)"



### Component Types
In simulations, we don't always simulate the entire chip at once. So there's a property `component_types` to view which combinations we've simulated.

It also includes a little blurb to help clarify what the names imply!

In [7]:
reader.component_types

╒═════════════╤════════════════════════════════════════════════╤════════════════════════════════════════════╕
│ QubitOnly   │ QubitCavity                                    │ QubitFeedline                              │
╞═════════════╪════════════════════════════════════════════════╪════════════════════════════════════════════╡
│ Qubit only  │ Qubit capacitvely coupled to a readout cavity. │ Qubit capacitvely coupled to a drive line. │
╘═════════════╧════════════════════════════════════════════════╧════════════════════════════════════════════╛


['QubitOnly', 'QubitCavity', 'QubitFeedline']

### Characteristics of Component Type

Within each `component_type`, the types of observables / component characteristics you can extract change. To see which characteristics are supported, we can use `.get_characteristic_info`.

In [4]:
reader.get_characteristic_info(component_type="QubitOnly");

╒═════════════════════════╤═══════════════════════════════════════╤═════════╤═══════════════╕
│ CSV Column Name         │ Description                           │ Units   │ Math Symbol   │
╞═════════════════════════╪═══════════════════════════════════════╪═════════╪═══════════════╡
│ Qubit_Frequency_GHz     │ Qubit 01 Transition Freq (Linear GHz) │ GHz     │ f_{q,01}      │
├─────────────────────────┼───────────────────────────────────────┼─────────┼───────────────┤
│ Qubit_Anharmonicity_MHz │ Qubit Anharmonicity (Linear MHz)      │ MHz     │ \alpha_{q}    │
╘═════════════════════════╧═══════════════════════════════════════╧═════════╧═══════════════╛


In [5]:
reader.get_characteristic_info(component_type="QubitCavity");

╒═════════════════════════╤════════════════════════════════════════════════╤═════════╤═══════════════╕
│ CSV Column Name         │ Description                                    │ Units   │ Math Symbol   │
╞═════════════════════════╪════════════════════════════════════════════════╪═════════╪═══════════════╡
│ Qubit_Frequency_GHz     │ Qubit 01 Transition Freq (Linear GHz)          │ GHz     │ f_{q,01}      │
├─────────────────────────┼────────────────────────────────────────────────┼─────────┼───────────────┤
│ Qubit_Anharmonicity_MHz │ Qubit Anharmonicity (Linear MHz)               │ MHz     │ \alpha_{q}    │
├─────────────────────────┼────────────────────────────────────────────────┼─────────┼───────────────┤
│ Cavity_Frequency_GHz    │ Cavity Resonant Freq (Linear MHz)              │ GHz     │ _{c}          │
├─────────────────────────┼────────────────────────────────────────────────┼─────────┼───────────────┤
│ Coupling_Strength_MHz   │ Coupling between qubit and cavity (Linear MHz